In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display

In [2]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .widget-label { 
        font-size: 24px !important;
    }
    .slider-value { 
        font-size: 24px !important;
    }
    .widget-hslider .widget-readout {
        font-size: 22px !important;
    }
</style>
"""))

In [82]:
grid_size = 2000
circle_radius = 400

def update_circle(angle_deg, frequency):
    r1x = 0
    r1y = -circle_radius/10
    r2x = 0
    r2y = circle_radius/10
    sampling_rate = 16000
    t = np.linspace(0, 6, sampling_rate*6)
    signal = np.sin(t*2*np.pi*frequency)
    start = 3
    duration = 2*(10**(-np.floor(np.log10(frequency))))
    end = start+duration
    speed_of_sound_in_water = 1500

    angle_rad = np.radians(angle_deg)
    circle_x = circle_radius * np.cos(angle_rad)
    circle_y = circle_radius * np.sin(angle_rad)
    dist_from_source_to_r1 = max(1, np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2)))
    dist_from_source_to_r2 = max(1, np.sqrt(((circle_x-r2x)**2) + ((circle_y-r2y)**2)))
    time_to_r1 = dist_from_source_to_r1 / speed_of_sound_in_water
    time_to_r2 = dist_from_source_to_r2 / speed_of_sound_in_water
    receive_signal1 = signal/(dist_from_source_to_r1**2)
    receive_signal2 = signal/(dist_from_source_to_r2**2)
    r1_inds_begin = np.floor((start-time_to_r1)*sampling_rate).astype(int)
    r1_inds_end = r1_inds_begin + np.ceil(duration*sampling_rate).astype(int)
    chunk_of_received_signal1_in_window = receive_signal1[r1_inds_begin:r1_inds_end]
    r2_inds_begin = np.floor((start-time_to_r2)*sampling_rate).astype(int)
    r2_inds_end = r2_inds_begin + np.ceil(duration*sampling_rate).astype(int)
    chunk_of_received_signal2_in_window = receive_signal2[r2_inds_begin:r2_inds_end]
    smallest_r_inds = min(len(chunk_of_received_signal1_in_window), len(chunk_of_received_signal2_in_window))
    chunk_of_received_signal1_in_window = chunk_of_received_signal1_in_window[:(smallest_r_inds)]
    chunk_of_received_signal2_in_window = chunk_of_received_signal2_in_window[:(smallest_r_inds)]
    summed_signal = chunk_of_received_signal1_in_window+chunk_of_received_signal2_in_window
    vrms_summed_fixed = np.sqrt(np.mean(np.square(summed_signal)))
    vrms_summed_fixed_dB = 20*np.log10(vrms_summed_fixed)

    angle_degrees = np.arange(-90, 90)
    angle_radians = np.radians(angle_degrees)
    circle_x_arr = circle_radius * np.cos(angle_radians)
    circle_y_arr = circle_radius * np.sin(angle_radians)
    dist_from_source_to_r1_arr = np.sqrt(((circle_x_arr-r1x)**2) + ((circle_y_arr-r1y)**2))
    dist_from_source_to_r1_arr[dist_from_source_to_r1_arr<=1] = 1
    dist_from_source_to_r2_arr = np.sqrt(((circle_x_arr-r2x)**2) + ((circle_y_arr-r2y)**2))
    dist_from_source_to_r2_arr[dist_from_source_to_r2_arr<=1] = 1
    time_to_r1_array = dist_from_source_to_r1_arr / speed_of_sound_in_water
    time_to_r2_array = dist_from_source_to_r2_arr / speed_of_sound_in_water
    receive_signal1_for_degrees = signal[:, np.newaxis] /(dist_from_source_to_r1_arr**2)
    receive_signal2_for_degrees = signal[:, np.newaxis] /(dist_from_source_to_r2_arr**2)
    r1_indices = ((start - time_to_r1_array) * sampling_rate).astype(int) + np.arange(int(duration * sampling_rate))[:, np.newaxis]
    r2_indices = ((start - time_to_r2_array) * sampling_rate).astype(int) + np.arange(int(duration * sampling_rate))[:, np.newaxis]
    chunk_of_received_signal1_in_window_degrees = receive_signal1_for_degrees[r1_indices, np.arange(angle_degrees.shape[0])]
    chunk_of_received_signal2_in_window_degrees = receive_signal2_for_degrees[r2_indices, np.arange(angle_degrees.shape[0])]
    summed_signal_degrees = chunk_of_received_signal1_in_window_degrees+chunk_of_received_signal2_in_window_degrees
    vrms_summed = np.sqrt(np.mean(np.square(summed_signal_degrees), axis=0))
    vrms_summed_dB = 20*np.log10(vrms_summed)

    fig = plt.figure(figsize=(15, 5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[4, 4, 2], wspace=0.3) 
    map_ax = plt.subplot(gs[0, 0])
    map_ax.set_xlim(-grid_size / 2, grid_size / 2)
    map_ax.set_ylim(-grid_size / 2, grid_size / 2)
    map_ax.scatter(circle_x, circle_y, edgecolors='red', facecolors='none', linewidths=2, s=50, label=f"Source")
    map_ax.scatter(r1x, r1y, color='green', s=50, label=f"Receiver #1")
    map_ax.scatter(r2x, r2y, color='blue', s=50, label=f"Receiver #2")
    map_ax.set_xlabel("X (meters)")
    map_ax.set_ylabel("Y (meters)")
    map_ax.set_title(f"Sound Reception (2 receivers; 1 source)")
    map_ax.legend()

    gs_signals = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.8)
    received_ax = plt.subplot(gs_signals[0])
    received_ax.plot((1000*(t+time_to_r1-start)), receive_signal1+1e-5, 'green', label='rcvr #1')
    received_ax.plot((1000*(t+time_to_r2-start)), receive_signal2-1e-5, 'blue', label='rcvr #2')
    received_ax.set_xlabel("Time (ms)")
    received_ax.set_ylabel("Amplitude")
    received_ax.set_title("Received voltage signal")
    received_ax.legend(loc='upper right')
    received_ax.grid(which='both')
    received_ax.set_xlim(0, 1000*duration)
    received_ax.set_ylim(-2e-5, 2e-5)

    summed_ax = plt.subplot(gs_signals[1])
    summed_ax.plot(1000*(t[r2_inds_begin:r2_inds_end]+time_to_r2-start), summed_signal, 'k')
    summed_ax.set_xlabel("Time (ms)")
    summed_ax.set_ylabel("Amplitude")
    summed_ax.set_title("Summed voltage signal")
    summed_ax.grid(which='both')
    summed_ax.set_xlim(0, 1000*duration)
    summed_ax.set_ylim(-2e-5, 2e-5)

    gs_beam_pattern = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs[0, 2], hspace=0.8)
    beam_pattern_ax = plt.subplot(gs_beam_pattern[0])
    beam_pattern_ax.set_title(f'Beam pattern')
    beam_pattern_ax.plot(vrms_summed_dB, angle_degrees, color='k')
    beam_pattern_ax.scatter(vrms_summed_fixed_dB, angle_deg, edgecolors='red', facecolors='w', linewidth=2, zorder=2)
    beam_pattern_ax.set_xlim(-120, -90)
    beam_pattern_ax.grid(which='both')
    beam_pattern_ax.set_xlabel(r'dB$V_{rms}$ (dB)')
    beam_pattern_ax.set_ylabel('Angle (°)')

    plt.show()

angle_slider = widgets.IntSlider(
    value=0, min=-90, max=90, step=1, description="Angle (°)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

frequency_slider = widgets.IntSlider(
    value=0, min=1, max=1e3, step=1, description="Frequency (Hz)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

interactive_plot = widgets.interactive(update_circle, angle_deg=angle_slider, frequency=frequency_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='1000px'), max=90, min=-…